<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/main/SNOWPACKforPatrollers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SNOWPACK Model Setup and Execution

This notebook guides you through the process of setting up and running the SNOWPACK model. It includes steps for installing necessary libraries, compiling the SNOWPACK and MeteoIO code, configuring the model, fetching meteorological data from historical weather models, and running SNOWPACK.

# The following cells below set up SNOWPACK AND MeteoIO and the PATH structure to run it. They should not be edited. Pressing the play button below will run them all.

In [34]:
# @title Install environment updates
!apt-get update
!apt-get install -y build-essential cmake git liblapack-dev numdiff

%pip install openmeteo-requests requests-cache retry-requests pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from pathlib import Path
import openmeteo_requests
import requests_cache
from retry_requests import retry
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
from typing import Optional



Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,855 kB]
Fetched 5,833 kB in 2s (3,699 kB

In [ ]:
# @title Install SNOWPACK via Binaries (Fast)
# 1. Upgrade libstdc++6 to support GLIBCXX_3.4.32 (required by Snowpack 3.7.0)
!add-apt-repository -y ppa:ubuntu-toolchain-r/test
!apt-get update
!apt-get install -y libstdc++6

# 2. Download official compiled binaries (Snowpack bundle includes MeteoIO)
!wget -O snowpack.deb https://gitlabext.wsl.ch/api/v4/projects/32/packages/generic/snowpack/3.7.0/Snowpack-3.7.0-x86_64.deb

# 3. Install it
!dpkg -i --force-overwrite snowpack.deb
!apt-get install -f -y

In [ ]:
# @title Verify SNOWPACK Installation
import os
!which snowpack
!snowpack --version

# The above cells have set up SNOWPACK and made it ready to run. The below cells create the files, collect data, and run the snowpack model. This generates snowprofiles with no snow to start the model from.

In [44]:
#@title SNOWPACK Configuration Files Generation
"""
Generate .sno and .ini configuration files for SNOWPACK simulations
"""

#@markdown ## Station Configuration
station_id = "watrous"  #@param {"type":"string"}
station_name = "watrous_E_NTL"  #@param {"type":"string"}
latitude = 39.71438  #@param {"type":"number"}
longitude = -105.84475  #@param {"type":"number"}
altitude = 11800  #@param {"type":"integer"}
altitude_unit = "feet"  # @param ["meters", "feet"]
timezone = -7  #@param {"type":"number"}
profile_date = "2024-11-01T00:00:00"  #@param {"type":"string"}
coord_sys = "UTM" #@param {"type":"string"}
coord_param = "13S" #@param {"type":"string"}

#@markdown ## Virtual Slopes Configuration
num_slopes = 5  #@param {"type":"number", "min":1, "max":10}
include_flat = True  #@param {"type":"boolean"}
default_slope_angle = 38.0  #@param {"type":"number", "min":0, "max":90}

#@markdown ## Slope Directions (degrees: 0=North, 90=East, 180=South, 270=West)
north_slope = True  #@param {"type":"boolean"}
east_slope = True  #@param {"type":"boolean"}
south_slope = True  #@param {"type":"boolean"}
west_slope = True  #@param {"type":"boolean"}
custom_directions = ""  #@param {"type":"string"} {description:"Comma-separated azimuth angles (e.g., 45,135,225,315)"}

#@markdown ## SNOWPACK Settings
meas_tss = "false"  #@param ["true","false"]
enforce_measured_snow_heights = "false"  #@param ["true","false"]
write_profiles = "true"  #@param ["true","false"]
write_timeseries = "false"  #@param ["true","false"]
write_snowpack = "false"  #@param ["true","false"]

#@markdown ## Output Configuration
sno_directory = "/content/input"  #@param {"type":"string"}
ini_directory = "/content/config"  #@param {"type":"string"}
generate_config_files = True  #@param {"type":"boolean"}

# @markdown ### SNOWPACK Run End Date
snowpack_end_date_input = "2025-04-01"  # @param {type:"date"}
snowpack_end_date = f"{snowpack_end_date_input}T00:00"

# For local testing - use current directory
import os
if not os.path.exists("/content"):
    sno_directory = "."
    ini_directory = "."

def generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle):
    """Generate list of virtual slopes based on user selections"""
    slopes = []

    # Add flat slope if requested
    if include_flat:
        slopes.append((0.0, 0.0))

    # Add cardinal direction slopes
    if north_slope:
        slopes.append((default_slope_angle, 0.0))
    if east_slope:
        slopes.append((default_slope_angle, 90.0))
    if south_slope:
        slopes.append((default_slope_angle, 180.0))
    if west_slope:
        slopes.append((default_slope_angle, 270.0))

    # Add custom directions
    if custom_directions.strip():
        try:
            custom_angles = [float(x.strip()) for x in custom_directions.split(',')]
            for angle in custom_angles:
                if 0 <= angle <= 360:
                    slopes.append((default_slope_angle, angle))
        except ValueError:
            print("Warning: Invalid custom directions format. Use comma-separated numbers.")

    return slopes

def to_meters(value: float, unit: str) -> float:
    """Normalize altitude inputs so downstream code always works in meters."""
    unit = unit.lower()
    if unit == "meters":
        return float(value)
    if unit == "feet":
        return float(value) * 0.3048
    raise ValueError(f"Unsupported altitude unit: {unit}")

# Using variables from the configuration cell
altitude_meters = to_meters(altitude, altitude_unit)

def create_sno_content(station_id, station_name, longitude, latitude, altitude_meters, timezone, profile_date, slope_angle, slope_azimuth):
    """Create .sno file content for a single slope"""

    content = f"""SMET 1.1 ASCII
[HEADER]
station_id       = {station_id}
station_name     = {station_name}
longitude        = {longitude}
latitude         = {latitude}
altitude         = {altitude_meters}
nodata           = -999
tz               = {timezone}
source           = OpenMeteo
prototype        = SNOWPACK
ProfileDate      = {profile_date}
HS_Last          = 0.0000
SlopeAngle       = {slope_angle}
SlopeAzi         = {slope_azimuth}
nSoilLayerData   = 0
nSnowLayerData   = 0
SoilAlbedo       = 0.09
BareSoil_z0      = 0.020
CanopyHeight     = 0.00
CanopyLeafAreaIndex = 0.00
CanopyDirectThroughfall = 1.00
ErosionLevel     = 0
TimeCountDeltaHS = 0.000000
WindScalingFactor = 1.00

fields           = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo
[DATA]
"""

    return content

def create_ini_content(station_id, filename, snowfiles, meas_tss, enforce_measured_snow_heights,
                        coord_sys, coord_param, timezone, write_profiles, write_timeseries, write_snowpack):
    """Create .ini file content with multiple SNOWFILE entries

    Args:
        station_id: Station identifier
        filename: SMET filename
        snowfiles: List of .sno filenames
        meas_tss: MEAS_TSS setting ("true" or "false")
        enforce_measured_snow_heights: ENFORCE_MEASURED_SNOW_HEIGHTS setting ("true" or "false")
        coord_sys: Coordinate system (e.g., "UTM")
        coord_param: Coordinate parameter (e.g., "13S")
        timezone: Timezone offset (e.g., -7)
        write_profiles: PROF_WRITE setting ("true" or "false")
        write_timeseries: TS_WRITE setting ("true" or "false")
        write_snowpack: SNOW_WRITE setting ("true" or "false")

    Returns:
        String containing .ini file content
    """

    content = f"""[General]
BUFFER_SIZE = 370
BUFF_BEFORE = 1.5

[Input]
COORDSYS = {coord_sys}
COORDPARAM = {coord_param}
TIME_ZONE = {timezone}

METEO = SMET
METEOPATH = ../input
METEOFILE1 = {filename}
"""

    # Add SNOWFILE entries for each slope
    for i, snowfile in enumerate(snowfiles, 1):
        content += f"SNOWFILE{i} = {snowfile}\n"

    content += f"""
[Output]
COORDSYS = {coord_sys}
COORDPARAM = {coord_param}
TIME_ZONE = {timezone}
METEOPATH = ./output
EXPERIMENT = res
SNOW_WRITE = {write_snowpack.lower()}

TS_WRITE = {write_timeseries.lower()}
TS_FORMAT = SMET
TS_START = 0.0
TS_DAYS_BETWEEN = 0.125
PROF_WRITE = {write_profiles.lower()}
PROF_FORMAT = PRO
AGGREG_PRF = false
PROF_START = 0.0
PROF_DAYS_BETWEEN = 0.125

[Snowpack]
MEAS_TSS = {meas_tss.lower()}
ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights.lower()}
FORCING = ATMOS
SW_MODE = INCOMING
MEAS_INCOMING_LONGWAVE = false
HEIGHT_OF_WIND_VALUE = 4.5
HEIGHT_OF_METEO_VALUES = 4.5
ATMOSPHERIC_STABILITY = MO_MICHLMAYR
ROUGHNESS_LENGTH = 0.002
CALCULATION_STEP_LENGTH = 15.0
CHANGE_BC = false
THRESH_CHANGE_BC = -1.0
SNP_SOIL = false
SOIL_FLUX = false
GEO_HEAT = 0.06
CANOPY = false

[SnowpackAdvanced]
FIXED_POSITIONS = 0.25 0.5 1.0 -0.25 -0.10
SNOW_EROSION = TRUE
WIND_SCALING_FACTOR = 1.0
NUMBER_SLOPES = {len(snowfiles)}
SNOW_REDISTRIBUTION = TRUE
    THRESH_RAIN = 1.4\n    T_CRAZY_MIN = 140\n    T_CRAZY_MAX = 360\n
[Filters]
ENABLE_METEO_FILTERS = true
PSUM::filter1 = min
PSUM::arg1::soft = true
PSUM::arg1::min = 0.0
TA::filter1 = min_max
TA::arg1::min = 240
TA::arg1::max = 320
RH::filter1 = min_max
RH::arg1::min = 0.01
RH::arg1::max = 1.2
RH::filter2 = min_max
RH::arg2::soft = true
RH::arg2::min = 0.05
RH::arg2::max = 1.0
ISWR::filter1 = min_max
ISWR::arg1::min = -10
ISWR::arg1::max = 1500
ISWR::filter2 = min_max
ISWR::arg2::soft = true
ISWR::arg2::min = 0
ISWR::arg2::max = 1500
RSWR::filter1 = min_max
RSWR::arg1::min = -10
RSWR::arg1::max = 1500
RSWR::filter2 = min_max
RSWR::arg2::soft = true
RSWR::arg2::min = 0
RSWR::arg2::max = 1500
ILWR::filter1 = min_max
ILWR::arg1::min = 188
ILWR::arg1::max = 600
ILWR::filter2 = min_max
ILWR::arg2::soft = true
ILWR::arg2::min = 200
ILWR::arg2::max = 400
TSS::filter1 = min_max
TSS::arg1::min = 200
TSS::arg1::max = 320
TSG::filter1 = min_max
TSG::arg1::min = 200
TSG::arg1::max = 320
HS::filter1 = min
HS::arg1::soft = true
HS::arg1::min = 0.0
HS::filter2 = rate
HS::arg2::max = 5.55e-5
VW::filter1 = min_max
VW::arg1::min = -2
VW::arg1::max = 70
VW::filter2 = min_max
VW::arg2::soft = true
VW::arg2::min = 0.0
VW::arg2::max = 50.0

[Interpolations1D]
MAX_GAP_SIZE = 86400
PSUM::resample1 = accumulate
PSUM::ACCUMULATE::PERIOD = 900
HS::resample1 = linear
HS::LINEAR::MAX_GAP_SIZE = 43200
VW::resample1 = nearest
VW::NEAREST::EXTRAPOLATE = true
DW::resample1 = nearest
DW::NEAREST::EXTRAPOLATE = true

[Generators]
ILWR::generator1 = AllSky_LW
ILWR::arg1::type = Carmona
ILWR::arg1::shade_from_dem = FALSE
ILWR::arg1::use_rswr = FALSE
ILWR::generator2 = ClearSky_LW
ILWR::arg2::type = Dilley
"""

    return content

def get_slope_filename(angle, azimuth, slope_index, include_flat):
    """Get filename for slope naming convention.

    Naming rules:
    - Flat slope (angle == 0.0): Returns 'keystone' (no number)
    - Non-flat slopes: Returns 'keystone1', 'keystone2', etc.
    - Numbering depends on whether flat slope is included:
      * If flat included: non-flat slopes start from index 1
      * If flat not included: all slopes start from index 1

    Args:
        angle: Slope angle in degrees (0.0 for flat)
        azimuth: Slope azimuth in degrees
        slope_index: Index of slope in the slopes list
        include_flat: Whether flat slope is included in the list

    Returns:
        Filename base (without .sno extension)
    """
    if angle == 0.0:
        return "keystone"  # Flat slope gets no number
    else:
        # For numbered slopes, determine the number based on position
        if include_flat:
            # If flat is included, non-flat slopes start from index 1
            return f"keystone{slope_index}"
        else:
            # If no flat, start numbering from 1
            return f"keystone{slope_index + 1}"

def main():
    """Main execution function"""

    print("SNOWPACK Configuration Files Generation")
    print("=" * 50)
    print(f"Station: {station_name} ({station_id})")
    print(f"Location: {latitude:.6f}°N, {longitude:.6f}°W, {altitude_meters}m")
    print(f"Sno Directory: {sno_directory}")
    print(f"Ini Directory: {ini_directory}")
    print("=" * 50)

    if not generate_config_files:
        print("Configuration generation disabled. Set 'generate_config_files' to True to generate files.")
        return

    # Validate profile_date format
    try:
        from datetime import datetime
        datetime.strptime(profile_date, "%Y-%m-%dT%H:%M:%S")
    except ValueError:
        print(f"Warning: profile_date format may be incorrect. Expected format: YYYY-MM-DDTHH:MM:SS")
        print(f"  Current value: {profile_date}")

    # Validate directories
    try:
        os.makedirs(sno_directory, exist_ok=True)
        os.makedirs(ini_directory, exist_ok=True)
    except Exception as e:
        print(f"Error creating directories: {e}")
        return

    # Generate slopes
    slopes = generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle)

    # Limit to requested number of slopes
    if len(slopes) > num_slopes:
        slopes = slopes[:num_slopes]
        print(f"Warning: Limited to {num_slopes} slopes as requested")

    # Create multiple .sno files and collect snowfile names
    snowfiles = []
    sno_files_created = []

    for i, (angle, azimuth) in enumerate(slopes):
        # Get filename based on new naming convention
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat) + ".sno"
        snowfiles.append(sno_filename)

        # Create .sno content for this slope
        sno_content = create_sno_content(station_id, station_name, longitude, latitude, altitude_meters, timezone, profile_date, angle, azimuth)

        # Write .sno file to input directory
        sno_filepath = os.path.join(sno_directory, sno_filename)
        try:
            with open(sno_filepath, "w") as f:
                f.write(sno_content)
            sno_files_created.append(sno_filepath)
        except Exception as e:
            print(f"Error writing {sno_filepath}: {e}")
            continue

    # Create .ini content with multiple SNOWFILE entries
    ini_content = create_ini_content(
        station_id,
        f"{station_id}.smet",
        snowfiles,
        meas_tss,
        enforce_measured_snow_heights,
        coord_sys,
        coord_param,
        timezone,
        write_profiles,
        write_timeseries,
        write_snowpack
    )

    # Write .ini file to keystone directory
    ini_filename = os.path.join(ini_directory, f"{station_id}.ini")
    try:
        with open(ini_filename, "w") as f:
            f.write(ini_content)
    except Exception as e:
        print(f"Error writing {ini_filename}: {e}")
        return

    # Display results
    print(f"\n Configuration files created:")
    print(f"   .ini file: {ini_filename}")
    print(f"   .sno files:")
    for sno_file in sno_files_created:
        print(f"     {sno_file}")

    print(f"\nVirtual slopes configured: {len(slopes)}")
    for i, (angle, azimuth) in enumerate(slopes):
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat)
        if angle == 0.0:
            print(f"   Slope {i+1}: Flat (0°) -> {sno_filename}.sno")
        else:
            direction = ""
            if azimuth == 0.0:
                direction = "North"
            elif azimuth == 90.0:
                direction = "East"
            elif azimuth == 180.0:
                direction = "South"
            elif azimuth == 270.0:
                direction = "West"
            else:
                direction = f"{azimuth}°"
            print(f"   Slope {i+1}: {angle}° slope facing {direction} -> {sno_filename}.sno")

    print(f"\nSNOWPACK settings:")
    print(f"   MEAS_TSS = {meas_tss}")
    print(f"   ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights}")
    print(f"   NUMBER_SLOPES = {len(slopes)}")

    print("\n🎉 SNOWPACK configuration generation complete!")

# Run the main function
if __name__ == "__main__":
    main()

SNOWPACK Configuration Files Generation
Station: watrous_E_NTL (watrous)
Location: 39.714380°N, -105.844750°W, 3596.6400000000003m
Sno Directory: /content/input
Ini Directory: /content/config

 Configuration files created:
   .ini file: /content/config/watrous.ini
   .sno files:
     /content/input/keystone.sno
     /content/input/keystone1.sno
     /content/input/keystone2.sno
     /content/input/keystone3.sno
     /content/input/keystone4.sno

Virtual slopes configured: 5
   Slope 1: Flat (0°) -> keystone.sno
   Slope 2: 38.0° slope facing North -> keystone1.sno
   Slope 3: 38.0° slope facing East -> keystone2.sno
   Slope 4: 38.0° slope facing South -> keystone3.sno
   Slope 5: 38.0° slope facing West -> keystone4.sno

SNOWPACK settings:
   MEAS_TSS = false
   ENFORCE_MEASURED_SNOW_HEIGHTS = false
   NUMBER_SLOPES = 5

🎉 SNOWPACK configuration generation complete!


In [45]:
# @title SMET Handling, API Setup, and Parameters

# @markdown ### Location Settings are used from above.
# Pulling from the configuration cell
# latitude = 39.56858687967004  # @param {type:"number"}
# longitude = -105.91900397453021  # @param {type:"number"}
# altitude_input = 3614  # @param {type:"number"}
# altitude_unit = "meters"  # @param ["meters", "feet"]

# @markdown ### Station Information is used from above.
# Pulling from the configuration cell
# station_name = "keystone_model"  # @param {type:"string"}

# @markdown ### Time Period
start_date = "2024-11-01"  # @param {type:"date"}
end_date = "2025-04-30"    # @param {type:"date"}

# @markdown ### Weather Model
model_selection = "ifs"  # @param ["nbm", "ifs", "gfs", "hrrr"]

# @markdown ### HS (Snow Depth) Source
hs_source = "model"  # @param ["model", "snodas"]

# @markdown ### Open-Meteo Elevation
openmeteo_elevation_mode = "use_selected_altitude"  # @param ["use_model_elevation", "use_selected_altitude"]

# @markdown ### SMET Generation
generate_files = True  # @param {type:"boolean"}

# --- Helper functions ----------------------------------------------------

# Imports for SNODAS
import struct
import tarfile
import gzip
import urllib.request
import urllib.error
from io import BytesIO
import datetime

def get_snodas_snow_depth(lat, lon, date_str, cache_dir="snodas_cache", debug=False):
    """
    Download and extract SNODAS snow depth from NSIDC.
    
    SNODAS Units:
    - Raw data: millimeters (mm) stored as integer (snow_depth_raw / 1000.0)
    - Returns: meters (m) as float
    
    This is different from Open-Meteo which returns centimeters (cm).
    """
    SNODAS_NODATA = -9999
    
    # Grid configurations (detected from file size)
    GRID_CONFIGS = {
        'old': {'XMIN': -124.73375000000000, 'YMAX': 52.87458333333333, 
                'XMAX': -66.94208333333333, 'YMIN': 24.94958333333333,
                'NCOLS': 6935, 'NROWS': 3351, 'name': 'Pre-Oct-2013'},
        'new': {'XMIN': -124.73333333333333, 'YMAX': 52.87500000000000,
                'XMAX': -66.94166666666667, 'YMIN': 24.95000000000000,
                'NCOLS': 3353, 'NROWS': 3353, 'name': 'Post-Oct-2013'}
    }
    
    # Check location bounds
    if lat < 24.95 or lat > 52.88 or lon < -124.74 or lon > -66.94:
        return None
    
    # Construct URL
    tar_filename = f"SNODAS_{date_str}.tar"
    data_base = "https://noaadata.apps.nsidc.org/NOAA/G02158/masked"
    year = date_str[:4]
    month = date_str[4:6]
    month_names = ["01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
                   "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec"]
    month_dir = month_names[int(month) - 1]
    data_url = f"{data_base}/{year}/{month_dir}/{tar_filename}"
    
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, tar_filename)
    
    try:
        # Download or use cache
        if os.path.exists(cache_path):
            with open(cache_path, 'rb') as f:
                tar_data = BytesIO(f.read())
        else:
            if debug:
                print(f"  Downloading {date_str}...")
            # Set a user-agent to avoid potential 403s
            req = urllib.request.Request(
                data_url, 
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            with urllib.request.urlopen(req, timeout=60) as response:
                tar_data = BytesIO(response.read())
                with open(cache_path, 'wb') as f:
                    f.write(tar_data.getvalue())
            tar_data.seek(0)
        
        # Extract and decompress
        with tarfile.open(fileobj=tar_data, mode='r') as tar:
            # Look for the .dat.gz file with the snow depth code (1036)
            snow_depth_gz_file = None
            for member in tar.getmembers():
                if '1036' in member.name and member.name.endswith('.dat.gz'):
                    snow_depth_gz_file = tar.extractfile(member)
                    break
            
            if snow_depth_gz_file is None:
                if debug: print("  Could not find snow depth file (1036) in tar archive")
                return None
        
            with gzip.open(snow_depth_gz_file, 'rb') as gz_file:
                data = gz_file.read()
        
        # Detect grid from file size
        num_values = len(data) // 2
        grid_config = None
        for config in GRID_CONFIGS.values():
            if num_values == config['NCOLS'] * config['NROWS']:
                grid_config = config
                break
        
        if grid_config is None:
            if debug: print(f"  Could not detect grid configuration for size {len(data)}")
            return None
        
        # Parse binary data
        SNODAS_NCOLS = grid_config['NCOLS']
        SNODAS_NROWS = grid_config['NROWS']
        values = struct.unpack(f">{SNODAS_NCOLS * SNODAS_NROWS}h", data)
        snow_depth_array = np.array(values).reshape((SNODAS_NROWS, SNODAS_NCOLS))
        
        # Calculate grid coordinates
        SNODAS_XMIN = grid_config['XMIN']
        SNODAS_YMAX = grid_config['YMAX']
        SNODAS_CELLSIZE_X = (grid_config['XMAX'] - SNODAS_XMIN) / SNODAS_NCOLS
        SNODAS_CELLSIZE_Y = (SNODAS_YMAX - grid_config['YMIN']) / SNODAS_NROWS
        
        col = int((lon - SNODAS_XMIN) / SNODAS_CELLSIZE_X)
        row = int((SNODAS_YMAX - lat) / SNODAS_CELLSIZE_Y)
        
        col = max(0, min(SNODAS_NCOLS - 1, col))
        row = max(0, min(SNODAS_NROWS - 1, row))
        
        # Extract value
        snow_depth_raw = snow_depth_array[row, col]
        if snow_depth_raw == SNODAS_NODATA or snow_depth_raw < 0:
            return None
        
        snow_depth_m = snow_depth_raw / 1000.0
        return snow_depth_m if snow_depth_m >= 0.0 else 0.0
            
    except Exception as e:
        if debug:
            print(f"  Error: {e}")
        return None



def resolve_openmeteo_elevation(mode: str, station_alt_m: float) -> Optional[float]:
    """
    Decide which elevation to send to the Open-Meteo API.
    - use_model_elevation     -> None (API uses its DEM; statistical downscaling stays enabled)
    - use_selected_altitude   -> the altitude value converted to meters above
    """
    mode = mode.lower()
    if mode == "use_model_elevation":
        return None
    if mode == "use_selected_altitude":
        return float(station_alt_m)
    raise ValueError(f"Unknown elevation mode: {mode}")


def create_smet_from_weather_data(weather_df: pd.DataFrame,
                                  output_path: str,
                                  station_id: str,
                                  station_name: str,
                                  latitude: float,
                                  longitude: float,
                                  altitude_meters: float,
                                  timezone: float = -7) -> None:
    """
    Create a SMET file from a weather DataFrame and write it to disk.

    Args:
        weather_df: DataFrame containing weather data with timestamp column
        output_path: Path where SMET file will be written
        station_id: Station identifier
        station_name: Station name
        latitude: Latitude in degrees
        longitude: Longitude in degrees
        altitude_meters: Altitude in meters
        timezone: Timezone offset (default: -7 for Mountain Time)
    """
    if "timestamp" in weather_df.columns:
        weather_df = weather_df.copy()
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
    else:
        weather_df = weather_df.copy()
        weather_df.reset_index(inplace=True)
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

    fields = ["timestamp", "TA", "RH", "TSG", "VW", "DW", "ISWR", "PSUM"]
    if "HS" in weather_df.columns:
        fields.append("HS")

    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_path, "w") as f:
        f.write("SMET 1.1 ASCII\n")
        f.write("[HEADER]\n")
        f.write(f"station_id = {station_id}\n")
        f.write(f"station_name = {station_name}\n")
        f.write(f"latitude = {latitude:.10f}\n")
        f.write(f"longitude = {longitude:.10f}\n")
        f.write(f"altitude = {altitude_meters}\n")
        f.write("nodata = -777\n")
        f.write(f"Tz = {timezone}\n")
        fields_str = "\t".join(fields)
        f.write(f"fields = {fields_str}\n")

        field_count = len(fields)
        units_offset = ["0"] * field_count
        units_multiplier = ["1"] * field_count
        f.write(f"units_offset     = {' '.join(units_offset)}\n")
        f.write(f"units_multiplier = {' '.join(units_multiplier)}\n")

        f.write("[DATA]\n")
        for _, row in weather_df.iterrows():
            values = []
            for field in fields:
                if field == "timestamp":
                    values.append(row[field].strftime("%Y-%m-%dT%H:%M:%S"))
                else:
                    val = row.get(field, -777)
                    if pd.isna(val):
                        val = -777
                    values.append(f"{val:.2f}")
            f.write("\t".join(values) + "\n")


def fetch_openmeteo_historical(lat: float,
                               lon: float,
                               start_date: str,
                               end_date: str,
                               model: str = "gfs_global",
                               elevation: Optional[float] = None) -> Optional[pd.DataFrame]:
    """
    Fetch historical forecast data from the Open-Meteo API.
    """
    cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "wind_direction_10m",
            "snow_depth",
            "direct_radiation",
            "precipitation",
        ],
        "models": model,
        "wind_speed_unit": "ms",
    }

    if elevation is not None:
        params["elevation"] = elevation

    print(f"Fetching {model} data for {lat}, {lon} from {start_date} to {end_date}")
    if "elevation" in params:
        print(f"Open-Meteo elevation parameter: {params['elevation']} (m)")

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
        print(f"Elevation: {response.Elevation()} m asl")

        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
        hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
        hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
        hourly_snow_depth = hourly.Variables(4).ValuesAsNumpy()
        hourly_direct_radiation = hourly.Variables(5).ValuesAsNumpy()
        hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()

        time_index = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        )

        df = pd.DataFrame(
            {
                "timestamp": time_index,
                "TA": hourly_temperature_2m + 273.15,   # Convert to Kelvin
                "RH": hourly_relative_humidity_2m / 100.0,
                "VW": hourly_wind_speed_10m,
                "DW": hourly_wind_direction_10m,
                "HS": hourly_snow_depth,
                "ISWR": hourly_direct_radiation,
                "PSUM": hourly_precipitation,
            }
        )
        # TSG (ground temperature) is not available from Open-Meteo API
        # Using assumed value of 273.15 K (0°C) as a reasonable default
        # SNOWPACK will calculate actual ground temperature during simulation
        df["TSG"] = 273.15
        df = df.replace([np.inf, -np.inf], np.nan)

        print(f"Fetched {len(df)} records")
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


# --- Derived values / summary -------------------------------------------

# Validate dates
try:
    from datetime import datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    if start_dt >= end_dt:
        raise ValueError(f"start_date ({start_date}) must be before end_date ({end_date})")
except ValueError as e:
    print(f"Date validation error: {e}")
    print("Please check your date inputs and try again.")
    raise

# Validate that altitude_meters is available (from configuration cell)
try:
    altitude_meters
except NameError:
    print("Error: altitude_meters not found. Please run the configuration cell first.")
    raise

# altitude_meters = altitude # Commented out direct assignment
openmeteo_elevation = resolve_openmeteo_elevation(openmeteo_elevation_mode, altitude_meters)

print("SMET Generation Parameters")
print("=" * 50)
print(f"Location: {latitude}, {longitude}")
print(f"Altitude input: {altitude} {altitude_unit} ({altitude_meters:.2f} m used for calculations)")
# print(f"Altitude: {altitude_meters:.2f} m")
print(f"Station: {station_name}")
print(f"Period: {start_date} to {end_date}")
print(f"Model: {model_selection}")
print(f"Generate: {generate_files}")
if openmeteo_elevation is None:
    print("Open-Meteo elevation: model elevation (automatic downscaling)")
else:
    print(f"Open-Meteo elevation: {openmeteo_elevation:.2f} m (selected altitude)")
print("=" * 50)

SMET Generation Parameters
Location: 39.71438, -105.84475
Altitude input: 11800 feet (3596.64 m used for calculations)
Station: watrous_E_NTL
Period: 2024-11-01 to 2025-04-30
Model: ifs
Generate: True
Open-Meteo elevation: 3596.64 m (selected altitude)


In [46]:
# @title Generate SMET Files
# @markdown Run this cell to generate SMET files with the parameters above

# Hardcoded output path
# output_directory = "/content/input" # Commented out hardcoded path

# Use sno_directory from the configuration cell
output_directory = sno_directory  # sno directory is input folder

# Model mapping
model_mapping = {
    "nbm": "ncep_nbm_conus",
    "ifs": "ecmwf_ifs",
    "gfs": "gfs_global",
    "hrrr": "gfs_hrrr"
}


if generate_files:
    print("Starting SMET generation...")

    # Parse model selection
    selected_models = [model_selection]

    # Validate and map models
    valid_models = ['nbm', 'ifs', 'gfs', 'hrrr']
    selected_models = [model for model in selected_models if model in valid_models]

    if not selected_models:
        print("No valid models selected!")
        print(f"Available models: {', '.join(valid_models)}")
    else:
        print(f"Processing {len(selected_models)} model(s): {', '.join(selected_models)}")
        print("="*50)

        successful_files = []

        for model in selected_models:
            print(f"\nProcessing {model}...")

            # Map to Open-Meteo model name
            openmeteo_model = model_mapping[model]

            # Fetch data
            df = fetch_openmeteo_historical(
                lat=latitude,
                lon=longitude,
                start_date=start_date,
                end_date=end_date,
                model=openmeteo_model,
                elevation=openmeteo_elevation
            )

            if df is None or df.empty:
                print(f"Failed to fetch data for {model}")
                continue

            # SNODAS Integration
            if 'hs_source' in locals() and hs_source == "snodas":
                print(f"  Fetching SNODAS snow depth for {len(df)} records...")
                # Determine unique dates to fetch
                df['date_key'] = df['timestamp'].dt.strftime("%Y%m%d")
                unique_dates = df['date_key'].unique()
                snodas_cache = {}
                
                print(f"  Retrieving SNODAS data for {len(unique_dates)} days...")
                for date_str in unique_dates:
                    try:
                        depth = get_snodas_snow_depth(latitude, longitude, date_str)
                        if depth is not None:
                            snodas_cache[date_str] = depth
                    except Exception as e:
                        print(f"    Warning: SNODAS fetch failed for {date_str}: {e}")
                
                # Map back to hourly data
                if snodas_cache:
                    # Create mapping series
                    snodas_series = df['date_key'].map(snodas_cache)
                    
                    # Count valid mapping
                    valid_count = snodas_series.count()
                    if valid_count > 0:
                        df['HS'] = snodas_series
                        # Fill missing with 0 or keep original? Using 0 if missing for now but warn
                        # Actually, if we want to fallback to model, we should only overwrite where valid
                        # BUT df['HS'] might not exist yet if model didn't provide it?
                        # OpenMeteo usually provides 'snow_depth' in meters if requested? 
                        # Wait, create_smet_from_weather_data looks for 'HS'. 
                        
                        print(f"  ✓ Applied SNODAS snow depth to {valid_count} records")
                    else:
                        print("  ⚠ No valid SNODAS data mapped")
                else:
                    print("  ⚠ No SNODAS data retrieved")
            
            # Create output filename with simplified naming
            output_file = os.path.join(output_directory, f"{station_id}.smet")
            # could add in _{model} if wanted to specify

            try:
                # Generate SMET file
                create_smet_from_weather_data(
                    weather_df=df,
                    output_path=output_file,
                    station_id=f"{station_id}_{model}",
                    station_name=f"{station_name}_{model}",
                    latitude=latitude,
                    longitude=longitude,
                    altitude_meters=altitude_meters,
                    timezone=timezone
                )

                print(f"SMET file created: {output_file}")
                print(f"Records: {len(df)}")
                print(f"Period: {df['timestamp'].min()} to {df['timestamp'].max()}")

                successful_files.append(output_file)

            except Exception as e:
                print(f"Error creating SMET for {model}: {e}")
                continue

        print(f"\nSuccessfully generated {len(successful_files)} SMET files:")
        for file in successful_files:
            print(f"  {file}")

        if successful_files:
            print(f"\nFiles saved to: {output_directory}")
else:
    print("Generation disabled. Set 'generate_files' to True to generate SMET files.")

Starting SMET generation...
Processing 1 model(s): ifs

Processing ifs...
Fetching ecmwf_ifs data for 39.71438, -105.84475 from 2024-11-01 to 2025-04-30
Open-Meteo elevation parameter: 3596.6400000000003 (m)
Coordinates: 39.68365478515625°N -105.875°E
Elevation: 3596.639892578125 m asl
Fetched 4344 records
SMET file created: /content/input/watrous.smet
Records: 4344
Period: 2024-11-01 00:00:00+00:00 to 2025-04-30 23:00:00+00:00

Successfully generated 1 SMET files:
  /content/input/watrous.smet

Files saved to: /content/input


In [ ]:
# @title Run this cell to run SNOWPACK
import subprocess
import sys

# Construct the path to the .ini file using variables from the configuration cell
ini_filepath = os.path.join(ini_directory, f"{station_id}.ini")

# Find the snowpack executable path dynamically
try:
    result = subprocess.run(["which", "snowpack"], capture_output=True, text=True, check=True)
    snowpack_executable_path = result.stdout.strip()
except (subprocess.CalledProcessError, FileNotFoundError):
    snowpack_executable_path = "snowpack"  # Fallback to PATH lookup

# Verify .ini file exists
if not os.path.exists(ini_filepath):
    raise FileNotFoundError(f"Configuration file not found: {ini_filepath}")

# Run SNOWPACK with enhanced error reporting
try:
    print(f"Running SNOWPACK with config: {ini_filepath}")
    result = subprocess.run(
        [snowpack_executable_path, "-c", ini_filepath, "-e", snowpack_end_date],
        cwd=ini_directory,
        check=True,
        capture_output=True,
        text=True
    )
    # If successful, print stdout
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"\nError running SNOWPACK (exit code {e.returncode})")
    print("="*40)
    print("SNOWPACK STDERR:")
    print(e.stderr)
    print("="*40)
    print("SNOWPACK STDOUT:")
    print(e.stdout)
    raise
except FileNotFoundError:
    print(f"SNOWPACK executable not found: {snowpack_executable_path}")
    print("Please ensure SNOWPACK is compiled and in PATH")
    raise


In [48]:
# @title Download .pro files and Open niViz
# @markdown # 5 files will be download.
# @markdown # 1 = N, 2=E, 3=S, 4=W,
import glob, shutil, os
from IPython.display import FileLink, Markdown

# Check if running in Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Determine temp directory based on environment
if IN_COLAB:
    temp_dir = "/content"
else:
    # For local environments, use current directory or a temp directory
    temp_dir = os.getcwd()

# Locate all .pro files
pro_files = glob.glob(os.path.join(ini_directory, 'output', '*.pro'))

if pro_files:
    # Copy all files to a simple location
    downloaded_files = []
    for pro_file in pro_files:
        filename = os.path.basename(pro_file)
        temp_file_path = os.path.join(temp_dir, filename)
        shutil.copy(pro_file, temp_file_path)
        downloaded_files.append(filename)

    # Create a zip file with all .pro files
    import zipfile
    zip_filename = 'snowpack_profiles.zip'
    zip_path = os.path.join(temp_dir, zip_filename)
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for filename in downloaded_files:
            file_path = os.path.join(temp_dir, filename)
            zipf.write(file_path, filename)

    # Download the zip file (only in Colab)
    if IN_COLAB:
        files.download(zip_path)
    else:
        print(f"Zip file created at: {zip_path}")
        print(f"Files included: {', '.join(downloaded_files)}")

    # Show how to open in niViz
    display(Markdown(f"""
    ---
    {len(downloaded_files)} SNOWPACK profile files have been downloaded as **`{zip_filename}`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    """))

    for filename in downloaded_files:
        display(Markdown(f"   - **`{filename}`**"))

else:
    display(Markdown("No .pro files found in the output directory."))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


    ---
    10 SNOWPACK profile files have been downloaded as **`snowpack_profiles.zip`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    

   - **`ksp_model_ifs4_res.pro`**

   - **`ksp_model_ifs1_res.pro`**

...

   - **`ksp_model_ifs2_res.pro`**

   - **`ksp_model_ifs_res.pro`**

   - **`watrous_ifs1_res.pro`**

   - **`watrous_ifs_res.pro`**

   - **`watrous_ifs4_res.pro`**

# This cell can erase input and ini files to start over.  

In [ ]:
# @title Clean up generated files
# @markdown Run this cell to remove generated .sno and .ini files.

run_cleanup = False #@param {type:"boolean"}

import os
import shutil

# Use variables from the configuration cell
sno_directory_to_clear = sno_directory
ini_directory_to_clear = ini_directory

def clear_directory(directory_path):
    """Removes all files and subdirectories within a given directory.

    Args:
        directory_path: Path to directory to clear

    Returns:
        tuple: (success: bool, files_deleted: int, errors: list)
    """
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return False, 0, []

    if not os.path.isdir(directory_path):
        print(f"Error: {directory_path} is not a directory")
        return False, 0, []

    # Count files before deletion
    file_count = len([f for f in os.listdir(directory_path)
                      if os.path.isfile(os.path.join(directory_path, f))])

    print(f"Clearing contents of: {directory_path}")
    print(f"  Found {file_count} file(s) and {len([d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))])} subdirectory(ies)")

    files_deleted = 0
    errors = []

    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
                files_deleted += 1
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                files_deleted += 1
        except Exception as e:
            error_msg = f"Failed to delete {item_path}: {e}"
            errors.append(error_msg)
            print(f"  Warning: {error_msg}")

    if errors:
        print(f"  Completed with {len(errors)} error(s)")
    else:
        print(f"  Successfully deleted {files_deleted} item(s)")

    return len(errors) == 0, files_deleted, errors

if run_cleanup:
    print("=" * 60)
    print("WARNING: This will delete all files in the following directories:")
    print(f"  - {sno_directory_to_clear}")
    print(f"  - {ini_directory_to_clear}")
    print("=" * 60)

    # Clear the input (sno) directory
    sno_success, sno_count, sno_errors = clear_directory(sno_directory_to_clear)

    # Clear the config (ini) directory
    ini_success, ini_count, ini_errors = clear_directory(ini_directory_to_clear)

    total_errors = len(sno_errors) + len(ini_errors)
    if total_errors == 0:
        print("\n✓ Cleanup complete successfully.")
    else:
        print(f"\n⚠ Cleanup completed with {total_errors} error(s).")
else:
    print("Cleanup is disabled. Check the 'Run cleanup' box to enable.")